In [1]:
#!pip install keras-tuner
#!pip install --upgrade tensorflow

import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
np.bool = np.bool_
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from kerastuner import HyperModel, RandomSearch
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import load_model
import seaborn as sns
from shapely.geometry import Point
import re
import random
import tensorflow as tf
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 26.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
    

<ipython-input-1-8ed8a0adeeae>:18: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel, RandomSearch


## Read trips dataset and external data

In [2]:
from google.colab import drive

# Mount the data path
drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/E-Scooter Capstone/Data'

Mounted at /content/drive


In [3]:
escooter_data = pd.read_csv(os.path.join(folder_path, 'start_demand_1H.csv'))

In [12]:
escooter_data.head(10)

,trip_start,Start Community Area Number,trip_count,bus_stops,train_stops,Trip Date,total_crimes,total_population,asian_population_percentage,black_population_percentage,...,median_household_income,low_income_percent,middle_income_percent,high_income_percent,not_hs_grad_percent,hs_grad_percent,some_college_percent,has_bachelors_percent,hs_grad_or_higher_percent,below_poverty_percent
0,2022-05-10 09:00:00,32,4,154,34.0,2022-05-10,7.0,39327,21.072037,7.079106,...,112045,0.132542,0.430534,0.436925,0.039708,0.032986,0.103038,0.824268,0.960,0.093567
1,2022-05-10 10:00:00,28,1,366,18.0,2022-05-10,19.0,62816,18.506431,24.218352,...,91907,0.287340,0.340899,0.371761,0.061334,0.098288,0.144424,0.695954,0.939,0.193814
2,2022-05-10 10:00:00,32,9,154,34.0,2022-05-10,7.0,39327,21.072037,7.079106,...,112045,0.132542,0.430534,0.436925,0.039708,0.032986,0.103038,0.824268,0.960,0.093567
3,2022-05-10 11:00:00,7,1,175,6.0,2022-05-10,3.0,69568,7.307958,3.789098,...,115907,0.173579,0.356590,0.469831,0.019613,0.042171,0.085229,0.852987,0.980,0.089959
4,2022-05-10 11:00:00,8,5,268,14.0,2022-05-10,18.0,98528,14.301518,7.167506,...,108578,0.184995,0.370865,0.444140,0.018511,0.050168,0.101568,0.829753,0.981,0.096262
5,2022-05-10 11:00:00,28,1,366,18.0,2022-05-10,19.0,62816,18.506431,24.218352,...,91907,0.287340,0.340899,0.371761,0.061334,0.098288,0.144424,0.695954,0.939,0.193814
6,2022-05-10 11:00:00,32,4,154,34.0,2022-05-10,7.0,39327,21.072037,7.079106,...,112045,0.132542,0.430534,0.436925,0.039708,0.032986,0.103038,0.824268,0.960,0.093567
7,2022-05-10 12:00:00,32,3,154,34.0,2022-05-10,7.0,39327,21.072037,7.079106,...,112045,0.132542,0.430534,0.436925,0.039708,0.032986,0.103038,0.824268,0.960,0.093567
8,2022-05-10 13:00:00,32,8,154,34.0,2022-05-10,7.0,39327,21.072037,7.079106,...,112045,0.132542,0.430534,0.436925,0.039708,0.032986,0.103038,0.824268,0.960,0.093567
9,2022-05-10 13:00:00,33,1,86,6.0,2022-05-10,6.0,26376,19.070367,22.490143,...,114631,0.187026,0.356776,0.456198,0.048025,0.051552,0.125833,0.774590,0.952,0.075755


LSTM Modeling

In [15]:
import os
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

random.seed(123)
np.random.seed(123)
tf.random.set_seed(123)

# Function to create sequences for LSTM
def create_sequences(data, target, seq_length, prediction_length):
    X, y = [], []
    for i in range(len(data) - seq_length - prediction_length + 1):
        X.append(data[i:i + seq_length])
        y.append(target[i + seq_length:i + seq_length + prediction_length])
    return np.array(X), np.array(y)

# Define sequence length and prediction length, can change if needed
sequence_length = 168
prediction_length = 24

additional_variables = ['temp', 'precip', 'snow', 'Gas Price', 'total_crimes']

# Sort data by time
escooter_data['trip_start'] = pd.to_datetime(escooter_data['trip_start'])
escooter_data.sort_values('trip_start', inplace=True)

# Define the split date
train_end_date = '2023-06-30'

# Split the data
train_data = escooter_data[escooter_data['trip_start'] <= train_end_date].copy()
test_data = escooter_data[escooter_data['trip_start'] > train_end_date].copy()

# Initialize scores dictionary
scores = {}

# List of community IDs
community_ids = range(1, 77) # Runs all 77 communities

def preprocess_and_train_model(train_data, test_data, additional_variables, sequence_length, prediction_length):
    # Power transformation stabilizes the variance
    power_transformer = PowerTransformer()
    train_data[additional_variables] = power_transformer.fit_transform(train_data[additional_variables])
    test_data[additional_variables] = power_transformer.transform(test_data[additional_variables])

    # Normalize trip_count using scaler
    scaler_trip_count = MinMaxScaler()
    train_data['trip_count_scaled'] = scaler_trip_count.fit_transform(train_data[['trip_count']])
    test_data['trip_count_scaled'] = scaler_trip_count.transform(test_data[['trip_count']])

    # Normalize additional variables using scaler
    scaler_additional = MinMaxScaler()
    train_data[additional_variables] = scaler_additional.fit_transform(train_data[additional_variables])
    test_data[additional_variables] = scaler_additional.transform(test_data[additional_variables])

    # Create sequences of features and target for training and testing
    X_train, y_train = create_sequences(train_data[additional_variables].values, train_data['trip_count_scaled'].values, sequence_length, prediction_length)
    X_test, y_test = create_sequences(test_data[additional_variables].values, test_data['trip_count_scaled'].values, sequence_length, prediction_length)

    # Reshape X arrays for LSTM input: (samples, time_steps, features)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], len(additional_variables)))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], len(additional_variables)))

    # Build model with dropout
    model = Sequential([
        LSTM(units=50, activation='relu', input_shape=(sequence_length, len(additional_variables)), dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
        LSTM(units=50, activation='relu', dropout=0.2, recurrent_dropout=0.2),
        Dense(units=prediction_length)  # Output layer for predicting the next 8 time steps of trip_count
    ])

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Fit model with early stopping
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)

    # Make predictions on test data
    test_predictions = model.predict(X_test)

    # Inverse transform predictions and actual values to get actual trip counts
    test_predictions = scaler_trip_count.inverse_transform(test_predictions.reshape(-1, prediction_length))
    y_test_actual = scaler_trip_count.inverse_transform(y_test.reshape(-1, prediction_length))

    # Calculate RMSE and MAE for each prediction step
    rmse = np.sqrt(mean_squared_error(y_test_actual, test_predictions))
    mae = mean_absolute_error(y_test_actual, test_predictions)

    return rmse, mae, y_test_actual, test_predictions

# Loop through each community and calculate RMSE and MAE
for community_id in community_ids:
    # Filter data for the chosen community
    train_data_community = train_data[train_data['Start Community Area Number'] == community_id].copy()
    test_data_community = test_data[test_data['Start Community Area Number'] == community_id].copy()

    # Check if there's enough data to create sequences
    if len(train_data_community) > sequence_length + prediction_length and len(test_data_community) > sequence_length + prediction_length:
        # Preprocess data and train model
        rmse, mae, y_test_actual, test_predictions = preprocess_and_train_model(train_data_community, test_data_community, additional_variables, sequence_length, prediction_length)
        scores[community_id] = {'RMSE': rmse, 'MAE': mae}

    else:
        scores[community_id] = None
        print(f"Community {community_id} does not have enough data for the sequence length and prediction length.")

# Print scores for all communities
for community_id, metrics in scores.items():
    if metrics is not None:
        print(f"Community {community_id}: RMSE = {metrics['RMSE']:.4f}, MAE = {metrics['MAE']:.4f}")
    else:
        print(f"Community {community_id}: Not enough data to calculate metrics.")


Community 74: Not enough data to calculate metrics.
